In [1]:
# 예시로 Jupyter Notebook에서 테스트하는 코드
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

df = pd.read_csv('../data/supplements_20250311.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40459 entries, 0 to 40458
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제조사     40459 non-null  object
 1   제품명     40459 non-null  object
 2   신고번호    40459 non-null  int64 
 3   등록일     40459 non-null  int64 
 4   유통기한    40457 non-null  object
 5   기능성     40451 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [6]:
# 키워드 목록 (사용자가 찾을 가능성이 높은 주요 기능성)
keywords = ['비타민C']

# 키워드별로 제품을 필터링 후 CSV로 저장
for keyword in keywords:
    filtered_df = df[df['기능성'].str.contains(keyword, case=False, na=False)]
    
    if not filtered_df.empty:
        file_name = f'preprocessed_{keyword.replace(" ", "_")}.csv'
        filtered_df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"📂 '{keyword}' 관련 제품을 '{file_name}'로 저장 완료!")


📂 '비타민C' 관련 제품을 'preprocessed_비타민C.csv'로 저장 완료!


In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CSV 파일 로드
df = pd.read_csv('preprocessed_비타민C.csv')

# 결측값 제거
df = df.dropna(subset=['기능성'])

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['기능성'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# **📌 키워드를 TF-IDF로 검색하는 방식으로 변경**
def recommend(keyword, limit=5):
    """키워드 기반 제품 추천"""
    # 🔹 키워드를 TF-IDF로 변환
    keyword_vec = tfidf.transform([keyword])

    # 🔹 키워드와 모든 제품 간 코사인 유사도 계산
    sim_scores = cosine_similarity(keyword_vec, tfidf_matrix)[0]

    # 🔹 유사도가 높은 상위 limit개 제품 가져오기
    top_indices = sim_scores.argsort()[-limit:][::-1]

    return df[['제품명']].iloc[top_indices]

# 예시 테스트: '비타민 C'와 관련된 추천 제품
recommend_results = recommend('비타민 C', limit=5)
print(recommend_results)


                 제품명
885   벨더웰 이뮨 멀티비타 듀오
2144    이뮨 듀오 멀티비타맥스
3363       간에좋은 밀크씨슬
1332     소중한 눈건강 루테인
468       멀티비타민 면역 업


In [16]:
# 키워드 목록 (사용자가 찾을 가능성이 높은 주요 기능성)
keywords = ['피로']

# 키워드별로 제품을 필터링 후 CSV로 저장
for keyword in keywords:
    filtered_df = df[df['기능성'].str.contains(keyword, case=False, na=False)]
    
    if not filtered_df.empty:
        file_name = f'preprocessed_{keyword.replace(" ", "_")}.csv'
        filtered_df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"📂 '{keyword}' 관련 제품을 '{file_name}'로 저장 완료!")

📂 '피로' 관련 제품을 'preprocessed_피로.csv'로 저장 완료!


In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CSV 파일 로드
df = pd.read_csv('preprocessed_피로.csv')

# 결측값 제거
df = df.dropna(subset=['기능성'])

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['기능성'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# **📌 키워드를 TF-IDF로 검색하는 방식으로 변경**
def recommend(keyword, limit=5):
    """키워드 기반 제품 추천"""
    # 🔹 키워드를 TF-IDF로 변환
    keyword_vec = tfidf.transform([keyword])

    # 🔹 키워드와 모든 제품 간 코사인 유사도 계산
    sim_scores = cosine_similarity(keyword_vec, tfidf_matrix)[0]

    # 🔹 유사도가 높은 상위 limit개 제품 가져오기
    top_indices = sim_scores.argsort()[-limit:][::-1]

    return df[['제품명']].iloc[top_indices]

# 예시 테스트: '피로'와 관련된 추천 제품
recommend_results = recommend('피로', limit=5)
print(recommend_results)


              제품명
178      간편한 밀크씨슬
125          티알에스
157       홍경천비타민C
184      골든루트밀크씨슬
136  피로엔 홍경천 밀크씨슬
